In [1]:
from torcheeg import transforms

transform = transforms.Compose([
    transforms.BandDifferentialEntropy(sampling_rate=200,
                                       band_dict={
                                           "delta": (1, 4),
                                           "theta": (4, 8),
                                           "alpha": (8, 13),
                                           "beta": (13, 30),
                                           "gamma": (30, 44)
                                       }),
    transforms.ToTensor()
])

In [87]:
import scipy.io as sio
import torch


def test(data_path, model_path):
    data = sio.loadmat(data_path)
    sample = data['EEG_ECClean']
    eeg_data = sample["data"][0][0][:, 2200:2400]

    x = transform(eeg=eeg_data)['eeg']
    x = torch.reshape(x, (1, 16, 5))

    model = torch.load(model_path, map_location=torch.device('cpu'))  # gpu训练时需要映射到cpu

    model.eval()
    with torch.no_grad():
        output = model(x)
    # print(output)
    print(output.argmax(1))
    return output.argmax(1)[0]

In [138]:
import scipy.io as sio
import torch

def test_v2(data_path, model_path):
    data = sio.loadmat(data_path)
    sample = data['EEG_ECClean']
    result = []
    for i in range(120):
        eeg_data = sample["data"][0][0][:, i * 200:(i + 1) * 200]

        x = transform(eeg=eeg_data)['eeg']
        x = torch.reshape(x, (1, 16, 5))

        model = torch.load(model_path, map_location=torch.device('cpu'))  # gpu训练时需要映射到cpu

        model.eval()
        with torch.no_grad():
            output = model(x)
            result.append(output.argmax(1)[0].tolist())
        # print(output)
        # print(output.argmax(1))
    return result

In [173]:
result_list = test_v2(r'C:\Users\bugs_\PycharmProjects\eegProject\data\Test_EEG\MDD\00106394_EEG_ECClean.mat',
                      r'C:\Users\bugs_\PycharmProjects\eegProject\torcheegProj\DGCNN_test\models\DE\DGCNN_9.pth')

In [174]:
from collections import Counter
result = Counter(result_list)
print(result)
maxTimes = max(result_list,key=result_list.count)
print('maxTimes', maxTimes)

Counter({1: 119, 0: 1})
maxTimes 1


In [53]:
import os

# 设置文件夹路径和类别名称
folders = [r'C:\Users\bugs_\PycharmProjects\eegProject\data\Test_EEG\HC_All',
           r'C:\Users\bugs_\PycharmProjects\eegProject\data\Test_EEG\MDD_All',
           r'C:\Users\bugs_\PycharmProjects\eegProject\data\Test_EEG\BD_All']
class_names = ['HC', 'MDD', 'BD']

# 创建存储数据集的字典
data_file = {'filename': []}

# 遍历每个类别的文件夹
for folder, class_name in zip(folders, class_names):
    # 获取文件夹中的Clean.mat文件 列表
    file_list = os.listdir(folder)
    file_list = [os.path.join(folder, file) for file in file_list if file.endswith('Clean.mat')]

    # 将数据加入到对应的数据集中
    data_file['filename'].extend([(file, class_name) for file in file_list])

In [66]:
model_path = r'C:\Users\bugs_\PycharmProjects\eegProject\torcheegProj\DGCNN_test\models\DE\DGCNN_8.pth'
label_dic = {'HC': 0, 'MDD': 1, "BD": 2}

total = 0
calculate = 0

for sub, label in data_file['filename']:
    if label == 'BD':
        total += 1
        # print(sub)
        output = test(sub, model_path)
        if output == label_dic['BD']:
            calculate += 1

tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([1])
tensor([0])
tensor([1])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([1])
tensor([1])
tensor([2])
tensor([1])
tensor([0])
tensor([0])
tensor([1])
tensor([2])
tensor([2])
tensor([2])
tensor([1])
tensor([1])
tensor([0])
tensor([0])
tensor([2])
tensor([2])
tensor([1])
tensor([0])
tensor([2])
tensor([2])
tensor([1])
tensor([1])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([1])
tensor([1])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])
tensor([2])


In [67]:
calculate / total

0.7246376811594203

In [22]:
import numpy as np

if output[0] == 2:
    print(1)